In [1]:
import pandas as pd

df = pd.read_csv('clean_churn_data_prepped.csv', index_col='Customer Number')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_monthlycharges_ratio,tenure_totalcharges_ratio
Customer Number,,,,,,,,,
0,1,0,0,0,29.85,29.85,0,0.033501,0.033501
1,34,1,1,1,56.95,1889.50,0,0.597015,0.017994
2,2,1,0,1,53.85,108.15,1,0.037140,0.018493
3,45,0,1,2,42.30,1840.75,0,1.063830,0.024447
4,2,1,0,0,70.70,151.65,1,0.028289,0.013188
...,...,...,...,...,...,...,...,...,...
7038,24,1,1,1,84.80,1990.50,0,0.283019,0.012057
7039,72,1,1,3,103.20,7362.90,0,0.697674,0.009779
7040,11,0,0,0,29.60,346.45,0,0.371622,0.031751


In [2]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [3]:
automl = setup(df, target='Churn')

,Description,Value
0,session_id,6783
1,Target,Churn
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(7032, 9)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [46]:
automl[7]

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_monthlycharges_ratio,tenure_totalcharges_ratio
Customer Number,,,,,,,,,
0,1,0,0,0,29.85,29.85,0,0.033501,0.033501
1,34,1,1,1,56.95,1889.50,0,0.597015,0.017994
2,2,1,0,1,53.85,108.15,1,0.037140,0.018493
3,45,0,1,2,42.30,1840.75,0,1.063830,0.024447
4,2,1,0,0,70.70,151.65,1,0.028289,0.013188
...,...,...,...,...,...,...,...,...,...
7038,24,1,1,1,84.80,1990.50,0,0.283019,0.012057
7039,72,1,1,3,103.20,7362.90,0,0.697674,0.009779
7040,11,0,0,0,29.60,346.45,0,0.371622,0.031751


In [47]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.7948,0.0000,0.4910,0.6651,0.5634,0.4332,0.4428,0.0080
lr,Logistic Regression,0.7942,0.8349,0.5325,0.6456,0.5826,0.4478,0.4521,0.4670
gbc,Gradient Boosting Classifier,0.7907,0.8393,0.5038,0.6448,0.5647,0.4297,0.4360,0.2000
lda,Linear Discriminant Analysis,0.7905,0.8280,0.5333,0.6345,0.5784,0.4405,0.4442,0.0130
catboost,CatBoost Classifier,0.7899,0.8378,0.5083,0.6410,0.5658,0.4297,0.4355,1.1510
ada,Ada Boost Classifier,0.7889,0.8349,0.5031,0.6394,0.5616,0.4255,0.4317,0.0840
lightgbm,Light Gradient Boosting Machine,0.7816,0.8267,0.5083,0.6169,0.5560,0.4132,0.4174,0.0550
rf,Random Forest Classifier,0.7735,0.8065,0.4955,0.5967,0.5406,0.3921,0.3956,0.1960
xgboost,Extreme Gradient Boosting,0.7733,0.8136,0.4902,0.5973,0.5380,0.3897,0.3934,0.4180
knn,K Neighbors Classifier,0.7607,0.7497,0.4396,0.5734,0.4968,0.3435,0.3492,0.0200


In [48]:
best_model

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=6783,
                solver='auto', tol=0.001)

In [49]:
df.iloc[-2:-1].shape

(1, 9)

In [50]:
predict_model(best_model, df.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_monthlycharges_ratio,tenure_totalcharges_ratio,Label
Customer Number,,,,,,,,,,
7041,4,1,0,1,74.4,306.6,1,0.053763,0.013046,0


In [51]:
save_model(best_model, 'RC')

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='Churn',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strate...
                 ('fix_perfect', Remove_100(target='Churn')),
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
          

In [52]:
import pickle

with open('RC_model.pk', 'wb') as f:
    pickle.dump(best_model, f)

In [53]:
with open('RC_model.pk', 'rb') as f:
    loaded_model = pickle.load(f)

In [55]:
new_data = df.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

ValueError: X has 8 features per sample; expecting 13

In [56]:
loaded_lda = load_model('RC')

Transformation Pipeline and Model Successfully Loaded


In [57]:
predict_model(loaded_lda, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,tenure_monthlycharges_ratio,tenure_totalcharges_ratio,Label
Customer Number,,,,,,,,,
7041,4,1,0,1,74.4,306.6,0.053763,0.013046,0


In [58]:
from IPython.display import Code

Code('predict_churn.py')

import pandas as pd
from pycaret.classification import predict_model, load_model

def load_data(filepath):
    """
    Loads churn data into a DataFrame from a string filepath.
    """
    df = pd.read_csv(filepath, index_col='customerID')
    return df


def make_predictions(df):
    """
    Uses the pycaret best model to make predictions on data in the df dataframe.
    """
    model = load_model('RC')
    predictions = predict_model(model, data=df)
    predictions.rename({'Label': 'Churn_prediction'}, axis=1, inplace=True)
    predictions['Churn_prediction'].replace({1: 'Churn', 0: 'No Churn'},
                                            inplace=True)
    return predictions['Churn_prediction']


if __name__ == "__main__":
    df = load_data('new_churn_data.csv')
    predictions = make_predictions(df)
    print('predictions:')
    print(predictions)

In [59]:
%run predict_churn.py

Transformation Pipeline and Model Successfully Loaded
predictions:
customerID
9305-CKSKC       Churn
1452-KNGVK    No Churn
6723-OKKJM    No Churn
7832-POPKP    No Churn
6348-TACGU    No Churn
Name: Churn_prediction, dtype: object
